In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [2]:
#load data
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [3]:
df = pd.read_csv("spam.csv")
df.head(25)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
df.shape

(5572, 2)

In [5]:
df.columns

Index(['Category', 'Message'], dtype='object')

In [6]:
df.drop_duplicates(inplace = True)

In [7]:
df.shape

(5157, 2)

In [8]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [9]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def process_text(Message):
    nopunc = [char for char in Message if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return [lemmatizer.lemmatize(stemmer.stem(word)) for word in words]

In [11]:
#show tokenisation by seperating words by commas
df['Message'].head(10).apply(process_text)

0    [go, jurong, point, crazi, avail, bugi, n, gre...
1                         [ok, lar, joke, wif, u, oni]
2    [free, entri, 2, wkli, comp, win, fa, cup, fin...
3        [u, dun, say, earli, hor, u, c, alreadi, say]
4    [nah, dont, think, goe, usf, live, around, tho...
5    [freemsg, hey, darl, 3, week, word, back, id, ...
6    [even, brother, like, speak, treat, like, aid,...
7    [per, request, mell, mell, oru, minnaminungint...
8    [winner, valu, network, custom, select, receiv...
9    [mobil, 11, month, u, r, entitl, updat, latest...
Name: Message, dtype: object

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df['Message']
y = df['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import LatentDirichletAllocation

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer=process_text)),
    ('lda', LatentDirichletAllocation(n_components=5, random_state=42)),
    ('classifier', MultinomialNB()) # Classifier is now the last step
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer=<function process_text at 0x7bce06aef250>)),
                ('lda',
                 LatentDirichletAllocation(n_components=5, random_state=42)),
                ('classifier', MultinomialNB())])

In [14]:
pred = pipeline.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         ham       0.87      1.00      0.93       896
        spam       0.00      0.00      0.00       136

    accuracy                           0.87      1032
   macro avg       0.43      0.50      0.46      1032
weighted avg       0.75      0.87      0.81      1032



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(analyzer=process_text)),
    ('classifier', MultinomialNB())
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer=<function process_text at 0x7bce06aef250>)),
                ('classifier', MultinomialNB())])

In [20]:
pred = pipeline.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.97       896
        spam       1.00      0.65      0.79       136

    accuracy                           0.95      1032
   macro avg       0.98      0.83      0.88      1032
weighted avg       0.96      0.95      0.95      1032

